# Exploratory Data Analysis
### Muf Tayebaly

In [1]:
#imports
import pandas as pd
import numpy as np
import pickle as pk
import string
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
import string
import re
from random import randint
from collections import defaultdict

In [2]:
#load data from pickle
corpus_data = pd.read_pickle('Data/muf_articles_corpus.pkl')
corpus_data.shape

(1914, 32)

In [3]:
corpus_data.describe()

,content_id,label,economy,topic2,science,mention_Trump,mention_Clinton,mention_Democrat,mention_Republican,mention_GOP,...,mention_border,mention_terrorist,mention_attack,mention_NRA,mention_sanctuary,mention_socialist,mention_fascist,mention_communist,mention_nationalist,mention_2A
count,1914.000000,1914.000000,1914.000000,1914.000000,1914.000000,1914.000000,1914.000000,1914.000000,1914.000000,1914.000000,...,1914.000000,1914.000000,1914.000000,1914.000000,1914.000000,1914.000000,1914.000000,1914.000000,1914.000000,1914.000000
mean,2147.275862,3.888192,0.253918,0.396552,0.349530,0.256008,0.027691,0.057994,0.071578,0.044410,...,0.003135,0.005747,0.022989,0.003657,0.002612,0.000522,0.001045,0.000522,0.002612,0.000522
std,794.150614,2.878083,0.435365,0.489309,0.476946,0.436541,0.164128,0.233793,0.257855,0.206057,...,0.055916,0.075611,0.149906,0.060380,0.051057,0.022858,0.032317,0.022858,0.051057,0.022858
min,60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1632.250000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2154.500000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2816.750000,7.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3384.000000,9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
corpus_data.head()

,content_id,content_body_clean,content_title_clean,label,economy,topic2,science,mention_Trump,mention_Clinton,mention_Democrat,...,mention_terrorist,mention_attack,mention_NRA,mention_sanctuary,mention_socialist,mention_fascist,mention_communist,mention_nationalist,mention_2A,content_source_desc
0,60,republican in congress be at a make or break m...,Tax reform now -- failure to act could mean mi...,9,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Fox News
1,61,the constitution authorize congress to tax ame...,Tax reform -- America needs a tax code that's ...,9,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Fox News
2,66,a longmothballed unfinished casinohotel on the...,Long-mothballed Las Vegas Strip casino-hotel s...,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fox News
3,80,if taxing foreign earnings that have already a...,"Trump Tax Plan May Free Up Corporate Dollars, ...",3,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,The New York Times
4,82,hurricane harvey have already take an enormous...,Hurricane Harvey is a humanitarian disaster. I...,5,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Vox


In [5]:
print("Count of articles by news source:")
corpus_data['content_source_desc'].value_counts()

Count of articles by news source:


Associated Press News    451
The New York Times       377
Fox News                 328
Vox                      303
Breitbart                268
100 Percent Fed Up       123
Media Matters             64
Name: content_source_desc, dtype: int64

In [6]:
print("Count of articles by label:")
corpus_data['label'].value_counts()

Count of articles by label:


0    318
2    220
3    219
5    217
7    210
1    183
4    174
8    137
9    133
6    103
Name: label, dtype: int64

## Topic Modeling

### Nouns only

In [8]:
from gensim import matutils, models
import scipy.sparse

In [11]:
dtm_data = pd.read_pickle('Data/muf_articles_atm.pkl')
dtm_data.head()

,aa,aaa,aaaah,aaas,aaron,aaronkleinshow,aaronson,aarp,aauw,ab,...,خلال,سمو,عددا,عسير,للمحافظة,متفقدا,مقرن,من,منصور,نائب
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
tdm = dtm_data.transpose()
tdm.head()

,0,1,2,3,4,5,6,7,8,9,...,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaaah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaas,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaron,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def nouns(text):
    #Given a string of text, tokenize the text and pull out only the nouns
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [12]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [ ]:
# Gensim requires dictionary
cv = pk.load(open("Data/cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())